In [132]:
#import the necessary liberay
import pandas as pd 
import os
import psycopg2
from sqlalchemy import create_engine
import pymysql
import csv
import datetime as datetime
import datetime as dt

In [85]:
#import the variout table 
df = pd.read_csv('make_details.csv')
df

,make_id,make_name,make_type
0,501,Aakron Xpress,Standard
1,502,ADLY,Standard
2,503,Alpha,Standard
3,504,Anglo,Standard
4,505,Aprilia,Standard
...,...,...,...
133,634,Volvo,Luxury
134,635,Voyager,Standard
135,636,Yamaha,Standard
136,637,Zephyr,Standard


In [86]:
location = pd.read_csv('locations.csv')
location 

,location_id,region,country,population,density
0,101,Northland,New Zealand,"201,500",16.11
1,102,Auckland,New Zealand,"1,695,200",343.09
2,103,Waikato,New Zealand,"513,800",21.50
3,104,Bay of Plenty,New Zealand,"347,700",28.80
4,105,Gisborne,New Zealand,"52,100",6.21
5,106,Hawke's Bay,New Zealand,"182,700",12.92
6,107,Taranaki,New Zealand,"127,300",17.55
7,108,Manawatū-Whanganui,New Zealand,"258,200",11.62
8,109,Wellington,New Zealand,"543,500",67.52
9,110,Tasman,New Zealand,"58,700",6.10


In [87]:
stolen_vehicle =pd.read_csv('stolen_vehicles.csv')
stolen_vehicle

,vehicle_id,vehicle_type,make_id,model_year,vehicle_desc,color,date_stolen,location_id
0,1,Trailer,623.0,2021.0,BST2021D,Silver,11/5/21,102
1,2,Boat Trailer,623.0,2021.0,OUTBACK BOATS FT470,Silver,12/13/21,105
2,3,Boat Trailer,623.0,2021.0,ASD JETSKI,Silver,2/13/22,102
3,4,Trailer,623.0,2021.0,MSC 7X4,Silver,11/13/21,106
4,5,Trailer,623.0,2018.0,D-MAX 8X5,Silver,1/10/22,102
...,...,...,...,...,...,...,...,...
4548,4549,NaN,NaN,NaN,NaN,NaN,2/18/22,102
4549,4550,NaN,NaN,NaN,NaN,NaN,2/14/22,109
4550,4551,NaN,NaN,NaN,NaN,NaN,3/9/22,102
4551,4552,NaN,NaN,NaN,NaN,NaN,3/7/22,109


In [88]:
#drop duplicate for each table and check for missing values 
make_car=df.drop_duplicates()
make_car.isnull().sum()

make_id      0
make_name    0
make_type    0
dtype: int64

In [89]:
#drop duplicate for each table and check for missing values 
stolen_vehicle =pd.read_csv('stolen_vehicles.csv')
stolen_vehicle=stolen_vehicle.drop_duplicates()
stolen_vehicle.isnull().sum()
stolen_vehicle=stolen_vehicle.dropna()
stolen_vehicle.isnull().sum()

vehicle_id      0
vehicle_type    0
make_id         0
model_year      0
vehicle_desc    0
color           0
date_stolen     0
location_id     0
dtype: int64

In [90]:
#drop duplicate for each table and check for missing values 
location = pd.read_csv('locations.csv')
location =location.drop_duplicates()
location.isnull().sum()

location_id    0
region         0
country        0
population     0
density        0
dtype: int64

In [98]:
# create a connect to your local host
import psycopg2

conn = None
try:
    print('Connecting to the database...')
    conn = psycopg2.connect(
        host='localhost',
        port=5432,
        dbname='bluewave',
        user='postgres',
        password='1'
    )
    cursor = conn.cursor()
    cursor.execute('SELECT version()')
    result = cursor.fetchone()
    print(result)
    cursor.close()
    conn.close()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed')


Connecting to the database...
('PostgreSQL 15.7, compiled by Visual C++ build 1939, 64-bit',)
Database connection closed


In [96]:
# create a connection to your database
import psycopg2

def load_data_into_database():
    connection = psycopg2.connect(
        user='postgres',
        password='1',
        dbname='bluewave',
        port=5432,
        host='localhost'
    )
    return connection
load_data_into_database()

<connection object at 0x0000026FF1247120; dsn: 'user=postgres password=xxx dbname=bluewave host=localhost port=5432', closed: 0>

In [148]:
# createa table 
import psycopg2

def connect_to_postgres():
    connection = psycopg2.connect(
        user='postgres',
        password='1',
        dbname='bluewave',  # Connect to the default 'postgres' database
        port=5432,
        host='localhost'
    )
    return connection


def load_data_table():
    conn = connect_to_postgres()
    cur = conn.cursor()
    query = '''
        DROP TABLE IF EXISTS location CASCADE;
        DROP TABLE IF EXISTS stolen_vehicle CASCADE;
        DROP TABLE IF EXISTS make_car CASCADE;

        CREATE TABLE IF NOT EXISTS location(
            location_id SERIAL PRIMARY KEY,
            region VARCHAR(250) NOT NULL,
            country VARCHAR(250) NOT NULL,
            population VARCHAR(256) NOT NULL,
            density FLOAT NOT NULL
        );

        CREATE TABLE IF NOT EXISTS make_car(
            make_id SERIAL PRIMARY KEY,
            make_name VARCHAR(250) NOT NULL,
            make_type VARCHAR(250) NOT NULL
        );

        CREATE TABLE IF NOT EXISTS stolen_vehicle(
            vehicle_id SERIAL PRIMARY KEY,
            vehicle_type VARCHAR(250) NOT NULL,
            make_id INT NOT NULL,
            model_year INT NOT NULL,
            vehicle_desc VARCHAR(500) NOT NULL,
            color VARCHAR(256) NOT NULL,
            date_stolen DATE NOT NULL,
            location_id INT NOT NULL,
            FOREIGN KEY(location_id) REFERENCES location(location_id),
            FOREIGN KEY(make_id) REFERENCES make_car(make_id)
        );
    '''
    cur.execute(query)
    conn.commit()
    cur.close()
    conn.close()


    # Step 1: Drop and create the new database
    
    
    # Step 2: Create the tables in the new database
load_data_table()
print('table create successfully')


table create successfully


In [149]:
# load data into the tables
def load_data_into_table(file):
    conn=connect_to_postgres()
    cur = conn.cursor()
    with open(file ,'r') as filename:
        reader= csv.reader(filename)
        next(reader)
        for row in reader:
            cur.execute('''
            insert into make_car(make_id,make_name,make_type) values(%s,%s,%s)
            ''', row)
        conn.commit()
        cur.close()
        conn.close()

filepath='make_details.csv'
load_data_into_table(filepath)
print('table is loaded sucessfully')
        
    

table is loaded sucessfully


In [ ]:
def load_data_into_table(file):
    conn=connect_to_postgres()
    cur = conn.cursor()
    with open(file ,'r') as filename:
        reader= csv.reader(filename)
        next(reader)
        for row in reader:
            cur.execute('''
            insert into location(location_id,region,country,population,density) values(%s,%s,%s,%s,%s)
            ''', row)
        conn.commit()
        cur.close()
        conn.close()

filepath='locations.csv'
load_data_into_table(filepath)
print('table is loaded sucessfully')

In [155]:
stolen_vehicle.to_csv('stolen_vehicle.csv', index=False)

In [ ]:

 stolen_vehicle['date_stolen'] = stolen_vehicle['date_stolen'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%y"))

In [142]:
stolen_vehicle['make_id']=stolen_vehicle['make_id'].astype(dtype='int')
stolen_vehicle['model_year']=stolen_vehicle['model_year'].astype(dtype='int')

In [156]:
def load_data_into_table(file):
    conn=connect_to_postgres()
    cur = conn.cursor()
    with open(file ,'r') as filename:
        reader= csv.reader(filename)
        next(reader)
        for row in reader:
            cur.execute('''
            insert into stolen_vehicle(vehicle_id,vehicle_type,make_id,model_year,vehicle_desc,color,date_stolen,location_id)\
            values(%s,%s,%s,%s,%s,%s,%s,%s)
            ''', row)
        conn.commit()
        cur.close()
        conn.close()

filepath='stolen_vehicle.csv'
load_data_into_table(filepath)
print('table is loaded sucessfully')

table is loaded sucessfully
